In [1]:
from utils import ingresos_financieros, calcular_cuota
import pandas as pd
from dateutil.relativedelta import relativedelta
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")
import numpy as np

In [2]:
def meses_entre_fechas(fecha_inicio, fecha_fin):
    """
    Calcula la distancia en meses entre dos fechas.

    :param fecha_inicio: Fecha de inicio (datetime)
    :param fecha_fin: Fecha de fin (datetime)
    :return: Número de meses entre las dos fechas
    """
    delta = relativedelta(fecha_fin, fecha_inicio)
    return delta.years * 12 + delta.months

In [ ]:
# cosechas = pd.read_csv(r"C:\Users\mariajose_chinchilla\Documents\GitHub\proyectos_varios\db\cosechas_junio.csv", sep=)

In [ ]:
# cosechas_df = cosechas[["NO_CREDITO", "CODIGO_CLIENTE", "ES_RECREDITO",
#                         "PRODUCTO", "CANTIDAD_DIAS_ATRASO", 
#                         "MONTO_DESEMBOLSADO",
#                         'rciInternoInicialDeudor', 'rciInternoFinalDeudor',
#                          'scoreDeudorInstallmentQualPacing']]

# cosechas_df.rename(columns={"rciInternoInicialDeudor": "RCI", 
#                             "rciInternoFinalDeudor": "RCI FINAL",
#                             "scoreDeudorInstallmentQualPacing": "SCORE"}, inplace=True)

In [ ]:
# recreditos = cosechas[(cosechas["ES_RECREDITO"] == "SI") & (cosechas["PRODUCTO"].isin(["SALUD", "LIQUIDEZ"]))]
# #recreditos["%pagado"] = 1 - cosechas_df["SALDO"] / cosechas_df["MONTO_DESEMBOLSADO"]

# pegar creditso cancelados por recredito

In [3]:
df = pd.read_excel(r"C:\Users\mariajose_chinchilla\Documents\GitHub\proyectos_varios\Salidas\espejo\Retrofit_espejo.xlsx")
cancelados = pd.read_csv(r"C:\Users\mariajose_chinchilla\Downloads\Recreditos.csv", sep=";")

In [4]:
cancelados = cancelados[(~cancelados["PLAZO_CRE_CANCEL"].isna()) & (cancelados["PLAZO_CRE_CANCEL"] != 0) & (~cancelados["incumple"].isna())]

In [5]:
cancelados["incumple"] = cancelados["incumple"].astype(str).str.replace("SI", "0")
cancelados["incumple"] = cancelados["incumple"].astype(str).str.replace("NO", "1")
cancelados["incumple"] = cancelados["incumple"].astype("int64")

In [6]:
cancelados["F_APER_CRE_CANCEL"] = pd.to_datetime(cancelados["F_APER_CRE_CANCEL"])
# Fecha objetivo a calcular la diferencia
fecha_fin = datetime(2024, 6, 30)

# Aplicar la función para calcular los meses activos
cancelados["MESES ACTIVOS"] = cancelados.apply(lambda x: meses_entre_fechas(x["F_APER_CRE_CANCEL"], fecha_fin), axis=1)
cancelados["INGRESOS RECIBIDOS"] = cancelados.apply(lambda x: ingresos_financieros(x["MONTO_CRED_CANCEL"],x["TASA_CAnt"]/1200, x["PLAZO_CRE_CANCEL"], x["MESES ACTIVOS"] - 1), axis=1)
cancelados["INGRESOS A 2024"] = cancelados.apply(lambda x: ingresos_financieros(x["MONTO_CRED_CANCEL"], x["TASA_CAnt"]/1200, x["PLAZO_CRE_CANCEL"], x["MESES ACTIVOS"] + 5), axis=1)

In [7]:
cancelados["INGRESOS RECIBIDOS"] = pd.to_numeric(cancelados["INGRESOS RECIBIDOS"], errors="coerce")
cancelados["INGRESOS A 2024"] = pd.to_numeric(cancelados["INGRESOS A 2024"], errors="coerce")

In [27]:
cancelados["CUOTA CREDITO"] = cancelados.apply(lambda x: calcular_cuota(x["MONTO_CRED_CANCEL"], x["TASA_CAnt"]/1200, x["PLAZO_CRE_CANCEL"]), axis=1)
cancelados["CAPITAL_PAGADO"] = cancelados["MONTO_CRED_CANCEL"] - (cancelados["MESES ACTIVOS"] - 1) * cancelados["CUOTA CREDITO"] + cancelados["INGRESOS RECIBIDOS"]
cancelados["MF CANCELADO"] = cancelados["INGRESOS A 2024"] - cancelados["INGRESOS RECIBIDOS"]
cancelados["FACTOR TASSA"] = cancelados["TASA_CAnt"] * cancelados["MONTO_CRED_CANCEL"]

In [28]:
cancelados_temp = cancelados.groupby(by="CRED_NUEVO").agg(
    MF_CANCELADO_TOTAL=("MF CANCELADO", "sum"),
    INCUMPLE_TOTAL=("incumple", "mean"),
    MONTO_CANCELADO_RECREDITO=("CAPITAL_PAGADO", "sum"),
    TPP_CANCEL=("FACTOR TASSA", "sum"),
    MONTO_ORIGINAL_CANCELADO_RECRED=("MONTO_CRED_CANCEL", "sum"),).reset_index()

In [29]:
cancelados_temp["FLAG 15%"] = np.where(cancelados_temp["INCUMPLE_TOTAL"] == 1, 1, 0)

In [35]:
a = cancelados.drop_duplicates(subset="CRED_NUEVO")
creds = len(a[a["RCI_global_final_CN"] > 50]) / len(a)
saldo = sum(a[a["RCI_global_final_CN"] > 50]["MONTO_CRED_NUEVO"]) / sum(a["MONTO_CRED_NUEVO"])

In [36]:
print(f"Proporción crédito es {creds} y de saldo es {saldo}")

Proporción crédito es 0.7849574431127323 y de saldo es 0.8461841897102166


In [30]:
cancelados_temp.to_excel(r"C:\Users\mariajose_chinchilla\Documents\GitHub\proyectos_varios\Salidas\espejo\cred_cancels.xlsx")